In [37]:
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel, Field
from IPython.display import display
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from typing import TypedDict, Annotated, List
import operator

In [38]:
class evaluationschema(BaseModel):
    feedback: str = Field(..., description="detailed feedback for essay")
    score: int = Field(..., description="score must be out of 10", ge=0, le=10)

In [ ]:

llm = HuggingFaceEndpoint(
    repo_id='Qwen/Qwen3-4B-Instruct-2507',
    huggingfacehub_api_token='hf_token',
    max_new_tokens=512
)
model = ChatHuggingFace(llm=llm)


In [40]:
parser = PydanticOutputParser(pydantic_object=evaluationschema)

In [41]:
prompt_template = PromptTemplate(
    input_variables=["text"],
    template="""
Analyze the following passage and return the result in the specified JSON schema.

{text}

{format_instructions}
""",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 4. Format the input
formatted_prompt = prompt_template.format(text="""
Environmental pollution is one of the most pressing problems of the modern world. 
It occurs when harmful substances contaminate air, water, and soil, causing damage to nature and human health. 
Air pollution from vehicles and factories leads to respiratory diseases and global warming. 
Water pollution from plastics and chemicals threatens aquatic life and safe drinking sources. 
Soil pollution reduces fertility and affects food production. 
Deforestation and industrialization worsen the problem, disturbing the balance of ecosystems. 
To protect the planet, people must adopt eco-friendly practices, reduce waste, and use renewable energy. 
A clean environment ensures a healthy, sustainable future.
""")


In [42]:
raw_output = model.invoke(formatted_prompt)
structured = parser.parse(raw_output.content)
display(structured)

evaluationschema(feedback='The passage provides a clear and comprehensive overview of environmental pollution, identifying its causes, effects on nature and human health, and proposing solutions such as eco-friendly practices, waste reduction, and renewable energy use. It logically connects various forms of pollution to real-world consequences and emphasizes the importance of sustainable living. The content is well-structured, informative, and relevant to current global issues. However, it could benefit from more specific examples or data to strengthen the argument. Overall, it is a strong and thoughtful discussion on environmental challenges and solutions.', score=9)

In [43]:
class upsclass(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_score: Annotated[list[int], operator.add]
    avg_score: float

In [44]:
def evaluate_language(state: upsclass) -> upsclass:
   prompt_template = PromptTemplate(
      input_variables=["essay"],
    template="""
evaluate the language quality of the following essay and provide a feedback and assign a score out of 10.
{essay}
{format_instructions}
""",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
   output = model.invoke(prompt_template.format(essay=state['essay']))
   structured = parser.parse(output.content)
   return {'language_feedback': structured.feedback, 'individual_score': [structured.score]}
   


In [45]:
def evaluate_analysis(state: upsclass) -> upsclass:
   prompt_template = PromptTemplate(
       input_variables=["essay"],
    template="""
evaluate the analysis quality of the following essay and provide a feedback and assign a score out of 10.
{essay}
{format_instructions}
""",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
   output = model.invoke(prompt_template.format(essay=state['essay']))
   structured = parser.parse(output.content)
   return {'analysis_feedback': structured.feedback, 'individual_score': [structured.score]}
   


In [46]:
def evaluate_clarity(state: upsclass) -> upsclass:
   prompt_template = PromptTemplate(
         input_variables=["essay"],
    template="""
evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10.
{essay}
{format_instructions}
""",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
   output = model.invoke(prompt_template.format(essay=state['essay']))
   structured = parser.parse(output.content)
   return {'clarity_feedback': structured.feedback, 'individual_score': [structured.score]}
   


In [47]:
def overall_feedback(state: upsclass) -> upsclass:
    prompt_template = PromptTemplate(
        input_variables=["essay", "language_feedback", "analysis_feedback", "clarity_feedback", "format_instructions"],
        template="""
Given the following essay and the feedback on language, analysis, and clarity, 
provide an overall feedback summary and an overall score out of 10.

Essay: {essay}
Language Feedback: {language_feedback}
Analysis Feedback: {analysis_feedback}
Clarity Feedback: {clarity_feedback}

{format_instructions}
""",
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    output = model.invoke(prompt_template.format(
        essay=state['essay'],
        language_feedback=state['language_feedback'],
        analysis_feedback=state['analysis_feedback'],
        clarity_feedback=state['clarity_feedback'],
    ))
    structured = parser.parse(output.content)
    overall_score = sum(state['individual_score']) // 3 if state['individual_score'] else 0
    return {'overall_feedback': structured.feedback, 'avg_score': overall_score}


In [48]:
graph = StateGraph(upsclass)

graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_clarity', evaluate_clarity)
graph.add_node('overall_feedback', overall_feedback)

graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_clarity')
graph.add_edge('evaluate_language', 'overall_feedback')
graph.add_edge('evaluate_analysis', 'overall_feedback')
graph.add_edge('evaluate_clarity', 'overall_feedback')
graph.add_edge('overall_feedback', END)

In [49]:
workflow = graph.compile()

In [51]:
initial_state = {
    'essay': """Environmental pollution is a major global issue caused by human activities such as industrialization, deforestation, and excessive use of vehicles. It contaminates air, water, and soil, harming human health, wildlife, and ecosystems. Reducing pollution through renewable energy, waste management, and sustainable practices is essential for protecting our planet’s future.""",
    'language_feedback': "",
    'analysis_feedback': "",
    'clarity_feedback': "",
    'overall_feedback': "",
    'individual_score': [],
    'avg_score': 0.0
}
final_state = workflow.invoke(initial_state)
display(final_state)

{'essay': 'Environmental pollution is a major global issue caused by human activities such as industrialization, deforestation, and excessive use of vehicles. It contaminates air, water, and soil, harming human health, wildlife, and ecosystems. Reducing pollution through renewable energy, waste management, and sustainable practices is essential for protecting our planet’s future.',
 'language_feedback': 'The essay presents a clear and concise overview of environmental pollution and its causes and effects. The language is straightforward and accurate, effectively conveying the main points. However, the content is superficial and lacks depth, examples, or specific details that would enhance clarity or engagement. There is no development of arguments, no evidence, or explanation of potential solutions beyond general statements. The structure is simple and logical, but it could benefit from more elaboration, transitions, or a stronger opening and closing. Overall, it meets the basic requir